In [258]:
from datetime import datetime
from datetime import date
from subgrounds.subgraph import SyntheticField
from subgrounds.subgrounds import Subgrounds
import pandas as pd

In [259]:
sg = Subgrounds()
snapshot = sg.load_api('https://hub.snapshot.org/graphql')

In [260]:
snapshot.Proposal.datetime = SyntheticField(
  lambda timestamp: str(datetime.fromtimestamp(timestamp)),
  SyntheticField.STRING,
  snapshot.Proposal.end,
)

In [261]:
proposals = snapshot.Query.proposals(
  orderBy='created',
  orderDirection='desc',
  first=10000,
  where=[
    snapshot.Proposal.space == 'fuse.eth',
    snapshot.Proposal.state == 'closed'
    ##snapshot.Proposal.title == 'OIP-18: Reward rate framework and reduction',
  ]
)

In [262]:
proposals_snapshots = sg.query_df([
    proposals.title,
    proposals.id,
    proposals.body,
    proposals.scores,
    proposals.scores_total
])

In [263]:
proposals_choices = sg.query(proposals.choices)

In [264]:
proposals_choices = pd.DataFrame(proposals_choices, columns = ['option_1', 'option_2', 'option_3', 'option_4', 'option_5','option_6','option_7','option_8','option_9','option_10','option_11','option_12','option_13','option_14','option_15','option_16','option_17','option_18','option_19','option_20','option_21','option_22'])

In [265]:
olympus_governance_view = pd.concat([proposals_snapshots,proposals_choices], axis=1)

In [266]:
##let's view the output just to make sure
olympus_governance_view

,proposals_title,proposals_id,proposals_body,proposals_scores_total,option_1,option_2,option_3,option_4,option_5,option_6,option_7,option_8,option_9,option_10,option_11,option_12,option_13,option_14
0,Whitelist GIVeth DAO as Fuse Pool creator,0xd75cab98b9143d7a16504dad01a3322ce6855369ff6c...,Summary:\nWe propose to whitelist GIVeth DAO a...,2.754376e+06,For Accept Proposal,Against Deny Proposal,None,None,None,None,None,None,None,None,None,None,None,None
1,Onboard VOLT to FeiRari DAO Pool # 8,0xd394651c02cda51cdab0ce717030042cf8a23d671581...,This proposal adds VOLT as a borrowable asset ...,1.013257e+07,For - add VOLT to Pool 8,Against - do nothing,None,None,None,None,None,None,None,None,None,None,None,None
2,Whitelist DeFiGeek Community as a Fuse Pool Cr...,0x10fe1c995aaa42fa42a9c9e8c26fe2b561e1d9488443...,Summary\nWe propose to whitelist DeFiGeek Comm...,1.675220e+01,For,Against,None,None,None,None,None,None,None,None,None,None,None,None
3,Whitelist StakeWise as a Fuse Pool Creator,0xbf3857c6e66640337170a93f74dbda91c57312aaf2b1...,Summary\n\nThis proposal is to whitelist Stake...,4.778143e-03,For,Against,None,None,None,None,None,None,None,None,None,None,None,None
4,Create Local Community,0xf388a3912a0ff5294c7cf68a964189617fad3c273e65...,Create local community in following countries\...,4.958574e+02,yes,no,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,[RIP-4] Staking Mechanism,QmPAA157pYkq3gUvM9LhGirGfe3nqnrCb5eT4hSEJs8gMs,*This assumes that RIP-1 and RIP-3 (or their d...,5.379841e+05,Create the staking mechanism,Explore other staking mechanism,None,None,None,None,None,None,None,None,None,None,None,None
141,[RIP-3] New Fee Structure,QmR26XEsCNjDgNbdqx9SCd8Rar1ShTN4E6aXr8yuvaWt8R,Summary:\nThis proposal aims to revise the cur...,6.542522e+05,Implement new fee structure,Keep old fees and fee structure,None,None,None,None,None,None,None,None,None,None,None,None
142,[RIP-2] Pool 2 With UNI-LP: RGT-ETH to Get to ...,QmR4mavGjBh639MRTKrjiiefj41FzHpcPXbMjayfqACy7J,*This will be used to bridge the gap between n...,2.752052e+05,Mint tokens for pool2 incentives,Do nothing for pool2 incentives,None,None,None,None,None,None,None,None,None,None,None,None
143,RIP-2 Alternative methods of driving value to ...,QmPeH4sFs6hpxeAN8r75JNbkJwqrDJfw6wke2j4Rr6Vuws,Rationale: Offer a better and more useful prod...,1.654471e+05,I support A only,I support B only,I support C only,I support D only,I support A&B only,I support A&C only,I support A&D only,I support B&C only,I support B&D only,I support C&D only,I support A&B&C only,I support A&B&D only,I support B&C&D only,I support ALL 4


In [267]:
path = '/Users/USER/Downloads/OIPS/fuse_proposals_table_'+str(date.today().strftime("%b-%d-%Y"))+'_'+str(len(olympus_governance_view))+'_proposals.csv'
olympus_governance_view.to_csv(path, index = False)

In [268]:
total_proposals = len(olympus_governance_view)
total_proposals

145

In [269]:
proposal_id = olympus_governance_view.iloc[0,1]

In [270]:
print(proposal_id)

0xd75cab98b9143d7a16504dad01a3322ce6855369ff6ca8d7b1685a4ac694a217


In [271]:

vote_tracker = snapshot.Query.votes(
orderBy = 'created',
orderDirection='desc',
first=10000,
where=[
  snapshot.Vote.proposal == proposal_id
]
)

In [272]:
voting_snapshots_list = sg.query_df([
    vote_tracker.id,
    vote_tracker.voter,
    vote_tracker.created,
    vote_tracker.choice,
    vote_tracker.vp
])

In [273]:
voting_snapshots_list

,votes_id,votes_voter,votes_created,votes_choice,votes_vp
0,0x8281ce505c29b6b019961f6e0e6e14fde7c5bc15d50f...,0x97b90FBc8904F861F76CB06BFa0A465b72C5E662,1650667132,1,1.185496e+05
1,0xa5d685dabf23e7ae3a60954e330a6106c26b41b40e5f...,0xC2634B9d0A0cFb756cDed34656E09828072A8a9B,1650628049,1,6.096007e+04
2,0xca7d2b7344087e3af22224271a927c3e6b857d8b0a34...,0x29687E2a2b38bbc1f4f9069f1f24F13a3c83d27D,1650568499,1,2.702278e+04
3,0x5a1e05b9a22922069534917a5a0b00cc4cf32dfb17a5...,0x1ccb144B700EC726d37Db38c617E154De6d9c0d0,1650420674,1,4.523094e+02
4,0x7d5245e24f681f0dfe15ca64913f29982630a9f79213...,0x839395e20bbB182fa440d08F850E6c7A8f6F0780,1650290002,1,2.173455e+03
5,0x0d63f77e2a10c397612f55858cc3a6f2510cdcb3c947...,0x8639F60e8F9b984eaf62B0bd89F62D6161e182F9,1650283481,1,2.545218e+06


In [274]:
voting_snapshots_list['Proposal'] = proposal_id
print(voting_snapshots_list)

                                            votes_id  \
0  0x8281ce505c29b6b019961f6e0e6e14fde7c5bc15d50f...   
1  0xa5d685dabf23e7ae3a60954e330a6106c26b41b40e5f...   
2  0xca7d2b7344087e3af22224271a927c3e6b857d8b0a34...   
3  0x5a1e05b9a22922069534917a5a0b00cc4cf32dfb17a5...   
4  0x7d5245e24f681f0dfe15ca64913f29982630a9f79213...   
5  0x0d63f77e2a10c397612f55858cc3a6f2510cdcb3c947...   

                                  votes_voter  votes_created  votes_choice  \
0  0x97b90FBc8904F861F76CB06BFa0A465b72C5E662     1650667132             1   
1  0xC2634B9d0A0cFb756cDed34656E09828072A8a9B     1650628049             1   
2  0x29687E2a2b38bbc1f4f9069f1f24F13a3c83d27D     1650568499             1   
3  0x1ccb144B700EC726d37Db38c617E154De6d9c0d0     1650420674             1   
4  0x839395e20bbB182fa440d08F850E6c7A8f6F0780     1650290002             1   
5  0x8639F60e8F9b984eaf62B0bd89F62D6161e182F9     1650283481             1   

       votes_vp                                           Pr

In [275]:
x=0
while x <total_proposals:
##print(olympus_governance_view.iloc[2,1])
    proposal_id = olympus_governance_view.iloc[x,1]

    vote_tracker = snapshot.Query.votes(
    orderBy = 'created',
    orderDirection='desc',
    first=10000,
    where=[
      snapshot.Vote.proposal == proposal_id
    ]
    )
    voting_snapshots = sg.query_df([
    vote_tracker.id,
    vote_tracker.voter,
    vote_tracker.created,
    vote_tracker.choice,
    vote_tracker.vp
    ])

    voting_snapshots['Proposal'] = proposal_id
    voting_snapshots_list=pd.concat([voting_snapshots_list, voting_snapshots])
    ##voting_snapshots['Proposal'] = proposal_id

#    pd.concat([voting_snapshots_list, voting_snapshots])
    x=x+1

In [276]:
len(voting_snapshots_list)

4389

In [277]:
path = '/Users/USER/Downloads/OIPS/fuse_votes_table_'+str(date.today().strftime("%b-%d-%Y"))+'_'+str(len(voting_snapshots_list))+'_votes.csv'
voting_snapshots_list.to_csv(path, index = False)